#### based on [perturb_all.py](https://github.com/HsiangHsu/rashomon-capacity/blob/main/awp/adult-compas-hsls/perturb-all.py)

In [2]:
import pandas as pd
import numpy as np
import pickle as pkl
from time import localtime, strftime
import time
import random
import argparse
import sys
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm

## accelerating
from itertools import islice
import multiprocessing
from multiprocessing import Pool

## custom packages
from training_MLP import *

In [ ]:
TEST_SPLIT = 0.30
NREPEAT = 1 # number of train-test splits
NMODELS = 1 # number of models per split
NNEURON = 20 # should be 100 or 200
NLAYER = 5
NEPOCH = 20
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TRAINLR = 0.001
MOMENTUM = 0.9

In [4]:
data_source = "data/obermeyer/obermeyer_data_cleaned.csv"
prediction_output = "predictions/obermeyer/awp_nn.csv"
target_variable = "cost_t"
features = ['dem_female', 'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1', 'dem_age_band_45-54_tm1',
            'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1', 'hypertension_elixhauser_tm1', 'cost_dialysis_tm1',
            'cost_emergency_tm1', 'cost_home_health_tm1', 'cost_ip_medical_tm1', 'cost_ip_surgical_tm1', 'cost_laboratory_tm1',
            'cost_op_primary_care_tm1', 'cost_op_specialists_tm1', 'cost_op_surgery_tm1', 'cost_other_tm1', 'cost_pharmacy_tm1',
            'cost_physical_therapy_tm1', 'cost_radiology_tm1', 'gagne_sum_tm1', 'person_id', 'threshold_25', 'threshold_50', 'threshold_75']

In [5]:
df = pd.read_csv(data_source)
X = df[features]
y = df[target_variable]

In [6]:
n = int(np.round(X.shape[0]*(1-TEST_SPLIT)))-1
nclass = len(set(y))
ntest = X.shape[0]-n

print(' X.shape = {}, y.shape = {}, nclass = {}\n'.format(X.shape, y.shape, nclass))
print(' X_train.shape = {}\n'.format(n))

 X.shape = (48784, 27), y.shape = (48784,), nclass = 963

 X_train.shape = 34148



In [7]:
base_train_loss = np.zeros((NREPEAT, ))
base_train_acc = np.zeros((NREPEAT, ))
base_test_loss = np.zeros((NREPEAT, ))
base_test_acc = np.zeros((NREPEAT, ))

rashomon_prob = np.zeros((NREPEAT, ntest, nclass, nclass))
rashomon_loss = np.zeros((NREPEAT, ntest, nclass))
sample_train_cap = np.zeros((NREPEAT, ntest))
y_train_all = np.zeros((NREPEAT, ntest))

In [8]:
def get_neural_network(X_train, y_train, X_test, random_seed, bootstrap_size=0.5):
    ## use random seed
    np.random.seed(random_seed)
    random.seed(random_seed)
    torch.manual_seed(random_seed)

    ## define network architexture
    nfeature = X_train.shape[1]
    nclass = 1 ## len(set(y_train))
    nn_arch = [nfeature, nclass, NNEURON, NLAYER]

    ## create and train model
    base_model = MLP(nn_arch).to(DEVICE)
    criterion = nn.MSELoss() ## nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(base_model.parameters(), lr=TRAINLR, momentum=MOMENTUM)
    base_model = train_model(base_model, X_train, y_train, NEPOCH, optimizer, criterion, DEVICE)

    ## get allocations
    base_model.eval()
    with torch.no_grad():
        outputs = base_model(torch.Tensor(X_test).to(DEVICE))  # Predictions/scores
        outputs_np = outputs.cpu().numpy()  # Detach and move to CPU as numpy array
    all_same = np.all(outputs_np == outputs_np[0, :], axis=0)
    print("Are all rows the same?", all_same.all()) 
    print(outputs_np)  # Print scores
    print(outputs_np.shape)
    return outputs_np[0, :]  # Return scores

In [9]:
output = []
for random_seed in range(NREPEAT):
    print("random seed", random_seed)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=random_seed)
    threshold_25 = X_test['threshold_25'].to_numpy()
    threshold_50 = X_test['threshold_50'].to_numpy()
    threshold_75 = X_test['threshold_75'].to_numpy()
    test_idx = X_test['person_id'].to_numpy()
    # print(test_idx.isna().sum())
    X_train = X_train.drop(columns=['person_id']).to_numpy()
    y_train = y_train.to_numpy()
    X_test = X_test.drop(columns=['person_id']).to_numpy()
    y_test = y_test.to_numpy()
    print(y_test.shape)

    predictions = []
    columns = []
    accuracy = []
    for i in tqdm(range(NMODELS)):
        columns.append(f'm_{i+1}')
        scores = get_neural_network(X_train, y_train, X_test, random_seed=i, bootstrap_size=0.5)
        predictions.append(scores)
        accuracy.append(float((np.round(scores)==y_test).sum() / len(y_test)))
    
    predictions_df = pd.DataFrame(predictions).transpose()
    predictions_df.columns=columns
    predictions_df["y"] = y_test
    predictions_df["idx"] = test_idx 
    predictions_df["seed"] = random_seed
    predictions_df['threshold_25'] = threshold_25
    predictions_df['threshold_50'] = threshold_50
    predictions_df['threshold_75'] = threshold_75
    output.append(predictions_df)
output = pd.concat(output)

random seed 0
(14636,)


100%|██████████| 1/1 [00:00<00:00,  1.86it/s]

Are all rows the same? False
[[-0.07817269]
 [-0.07793552]
 [-0.07844877]
 ...
 [-0.07863903]
 [-0.07829844]
 [-0.07865667]]
(14636, 1)


ValueError: Length of values (14636) does not match length of index (1)

In [ ]:
output.head()